In [2]:
!pip install git+https://git.ffmpeg.org/ffmpeg.git
!pip install ffprobe-python
!pip install pydub

: 

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
data_dir='C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/final_project_data/fma_small'
file_path='C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/final_project_data/fma_small/000/000002.mp3'
output_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_combined_mpwavvac.csv'

# Paths to the datasets
# tracks_path = '/Users/dkamboj6/Recitation/460j_fma_dataset/fma_metadata/tracks.csv'
# genres_path = '/Users/dkamboj6/Recitation/460j_fma_dataset/fma_metadata/genres.csv'
tracks_path=f"C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/final_project_data/fma_metadata/tracks.csv"
genres_path= f"C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/final_project_data/fma_metadata/genres.csv"
# Load data
tracks = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
genres = pd.read_csv(genres_path)


In [ ]:
file_path = r"C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/final_project_data/fma_small/000/000002.mp3"
print("Checking file path:", file_path)
print("File exists:", os.path.exists(file_path))

# List files in the directory to verify path and file names
print("Files in directory:")
for file in os.listdir(os.path.dirname(file_path)):
    print(file)


In [ ]:
import torchaudio
from pydub import AudioSegment
from transformers import Wav2Vec2Processor, Wav2Vec2Model
def extract_all_features(file_path):
    try:
        # Check if the file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"No file found at {file_path}")

        audio_segment = AudioSegment.from_mp3(file_path)
        samples = audio_segment.set_frame_rate(16000).set_channels(1).get_array_of_samples()
        wav_samples = np.array(samples).astype(np.float32)

        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

        inputs = processor(wav_samples, sampling_rate=16000, return_tensors="pt", padding=True)

        with torch.no_grad():
            outputs = model(inputs.input_values)
        features = outputs.last_hidden_state.squeeze().numpy()

        track_id = int(os.path.splitext(os.path.basename(file_path))[0])
        return features, track_id
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None, None
extract_all_features_file=extract_all_features(file_path)


In [ ]:

def extract_features(file_path):
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=None)
        
        # Get the Mel-frequency cepstral coefficients
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfccs_processed = np.mean(mfcc.T,axis=0)
        
        # Get the spectral contrast
        stft = np.abs(librosa.stft(y))
        contrast = librosa.feature.spectral_contrast(S=stft, sr=sr)
        contrast_processed = np.mean(contrast.T,axis=0)
        
        # Get the chroma feature
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_processed = np.mean(chroma.T,axis=0)
        
        # Get the mel-scaled spectrogram
        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_processed = np.mean(mel.T,axis=0)

        # Concatenate all features into one array
        features = np.hstack([mfccs_processed, contrast_processed, chroma_processed, mel_processed])
        
    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, error: {e}")
        return None 
    
    return features


In [ ]:
def load_genre_metadata(tracks_path):
    # Load track metadata for genre labels
    tracks_df = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
    genre_labels = tracks_df['track']['genre_top']  # Adjust if the column names are different
    return genre_labels.to_dict()

# Define the path to your tracks.csv metadata file
genre_dict = load_genre_metadata(tracks_path)



In [ ]:
def process_all_audio_files(base_directory):
    features_list = []
    track_ids = []

    # Walk through all files in the directory structure
    for subdir, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.mp3'):

                file_path = os.path.join(subdir, file).replace('\\','/')

                features = extract_all_features(file_path)
                if features is not None:
                    features_list.append(features)
                    # Extract track ID from filename
                    track_id = int(file.split('.')[0])
                    track_ids.append(track_id)

    return features_list, track_ids

In [ ]:
# Process all audio files and collect features
features, track_ids = process_all_audio_files(data_dir)

# Optionally, load the metadata and merge it with the extracted features
tracks_path = '/Users/dkamboj6/Recitation/460j_fma_dataset/fma_metadata/tracks.csv'
tracks = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
track_genres = tracks['track', 'genre_top']

# Create DataFrame
features_df = pd.DataFrame(features, index=track_ids)
features_df['genre'] = features_df.index.map(track_genres.get)

In [ ]:
display(features_df)

In [ ]:
# Save the DataFrame to a CSV file
features_df.to_csv(output_path)

print(f"All features with genre saved to {output_path}")
